<a href="https://colab.research.google.com/github/ohhongseok/ML-DeepLearning/blob/main/5/5-3/%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 핵심 키워드
 - 앙상블 학습
 - 랜덤 포리스트
 - 엑스트라 트리
 - 그레이디언트 부스팅

# 정형데이터와 비정형 데이터, 반정형 데이터
 - 정형 데이터 > 앙상블 학습이 가장 뛰어남 
 > 어떠한 구조로 되어있는 데이터셋으로, 행과 열로 이루어져 표에 저장할 수 있는 데이터
 - 비정형 데이터 > 신경망 알고리즘이 가장 뛰어남 
 > 그림, 음성파일 등 표로 데이터를 표현하기 어려운 데이터를 비정형 데이터라 한다.
 - 반정형 데이터
  > HTML / XML 같은 스크립트 언어로 구성되어 있는 데이터를 반정형 데이터라 한다.

# 앙상블 학습
 - 여러개의 분류기를 생성하고, 그 예측을 결합하여 보다 정확한 예측을 도출하는 기법
 - 강력한 하나의 모델을 사용하는것이 아닌, 약한 모델 여러개를 조합하여 더 정확한 예측에 도움을 줌

#랜덤 포리스트
 - 다수의 결정트리를 랜덤하게 만들어 결정트리 숲을 만듦
 - 각 결정트리의 예측결과를 사용해 최종 예측을 만듦
 - 사용하는 이유는 결정트리가 훈련데이터에 오버피팅 되는 경향이 있기 때문에 사용함
  > 예시.<br>
  건강의 위험도를 예측하기 위해선 많은 요소들을 고려해야 함. 하지만 수많은 class들을 통해 결정트리를 생성하게 되면, <br>가지의 수도 많아지고, 이는 오버피팅의 결과로 이어지게 됨 따라서 수많은 clss중 5개의 특징으로 첫번째 나무를 만들고,<br> 또 5개의 특징을 랜덤으로 뽑아 두번째 나무를 만드는 과정을 반복하여 결정트리를 만들면 하나의 숲이 만들어 진다.<br>
  여기에 여러 결정트리들이 내린 예측 값들 중 가장 많이 나온 값을 최종 예측값으로 정함
 - 랜덤포리스트 트리 훈련
  > - 데이터에서 랜덤하게 샘플을 추출하여 훈련데이터를 만들고, 한 샘플은 중복되어 추출될 수 있음 > 부트스트랩 샘플<br>
  - 부트스트랩 샘플은 훈련세트와 크기(개수)가 같게 추출됨
  - 랜덤포리스트의 각 트리들의 각 노드를 분할 할 때에는 전체 특성 중 일부 특성을 랜덤하게 뽑아 최선의 분할을 찾음
  - 일반적으로는 한 트리당 전체 특성의 제곱근 갯수만큼 특성을 선택함
  - 분류일 경우 `RandonForestClassifier`을, 회귀일 경우 `RandonForestRegressor`을 사용한다.  
  >> - 분류일 경우 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측함<br>
  >>- 회귀일 경우 각 트리의 예측을 평균함


In [1]:
# 사이킷런의 랜덤포리스트 분류 문제 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)

In [2]:
# RandomForestClassifier 교차검증 진행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1,random_state=42)

score = cross_validate(rf, train_input,train_target,
                       return_train_score=True,n_jobs=-1)
print(np.mean(score['train_score']),np.mean(score['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
# 랜덤포리스트는 결정트리가 제공하는 중요한 매개변수를 모두 사용할 수 있음
# 각 특성별 중요도를 확인해보자
rf.fit(train_input,train_target)
print(rf.feature_importances_)
# 앞장의 결정트리의 특성별 중요도와는 다른 값을 갖는다.
# 앞장의 결정트리는 당도가 매우 높게 측정 되었는데(5-2 결정트리 특성 중요도 참고)
# 여기에서는 상대적으로 고르게 분포됨
# 이를통해 하나의 특성에 과도하게 집중하지 않고, 좀 더 많은 특성이 훈련에 기여할 기회를 제공함
# 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 됨

[0.23167441 0.50039841 0.26792718]


In [4]:
# 랜덤포리스트는 중복을 허용한 부트스트랩 샘플을 사용함
# 이때 부트스트랩에 포함되지 않은 데이터들이 있는데 이런 샘플을 OOB 샘플이라 한다.
# 이렇게 남는 샘플을 활용하여 모델을 검증할 수 있음

# RandomForestClassifier 의 oob_score를 True로 해야 각 결정트리의 OOB 점수를
# 평균하여 출력함
# 디폴트로 총 100개의 tree를 생성
rf = RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_) # 출력된 100개의 결정트리의 OOB Score 평균값

0.8934000384837406


# 엑스트라 트리
 - 랜덤포리스트와 비슷한 앙상블 학습에 사용되는 결정트리 모델
 - 마찬가지로 기본 100개의 결정트리로 학습, 및 결정트리가 제공하는 대부분의 매개변수 사용 가능
 - 전체특성중 일부 특성을 랜덤하게 선택하여 트리 생성(랜덤포리스트와 똑같음)
 - 차이는 트리의 샘플 선택 시 랜덤포리스트와 다르게 부트스트랩 샘플을 사용하지 않음. 즉, 각 결정트리를 만들 때 ***전체의 데이터셋을 사용함***

# 랜덤포리스트와의 차이
 - 부트스트랩 샘플의 유무
 - 클래스 Split 시 랜덤포리스트는 변수에 대한 정보이득을 계산 후, 가장 높은 정보 이득을 가지는 변수를 활용, 따라서 연산량이 많이 필요함
 - 이에반해 엑스트라 트리는 랜덤하게 선택하기 때문에 랜덤포리스트보다 빠른 속도로 연산이 가능함

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(et,train_input,train_target,
                        return_train_score=True,n_jobs=-1)
# 엑스트라 트리의 교차검증 점수 확인
print(np.mean(score['train_score']),np.mean(score['test_score']))
# 랜덤포리스트와 비슷한 결과가 나옴
# 일반적으로 엑스트라 트리의 무작위성이 더 크기 때문에
# 랜덤포리스트보다 더 많은 결정트리를 훈련해야 한다.

0.9973541965122431 0.8905151032797809


In [6]:
# 엑스트라 트리의 특성 중요도
et.fit(train_input,train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그래디언트 부스팅
 - 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
 - 기본적으로 깊이가 3인 결정트리를 100개 사용함
 - 깊이가 앝은 트리를 사용하기에 과대적합에 강하고, 많은 트리를 사용하기에 높은 일반화 성능을 기대할 수 있음
 - 경사하강법을 활용하며, 결정트리를 계속 추가해 가며 손실함수의 가장 낮은 부분으로 이동한다.
 - 사이킷런의 `GradientBoostingClassifier`를 사용한다.

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 확인 결과 과대적합에 매우 강함을 알 수있음
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [10]:
# 데이터 세트의 갯수를 늘리고 학습률을 증가시키면 성능이 좀 더 향상됨
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)
# 학습 후 특성을 확인해 본 결과, 당도에 좀 더 집중함을 알 수 있음

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그래디언트 부스팅
 - 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘
 